In [2]:
import warnings

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
#import wqet_grader
from IPython.display import VimeoVideo
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.utils.validation import check_is_fitted

In [ ]:
# Task 2.2.1: Use your wrangle function to create a DataFrame frame1 from the CSV file data/buenos-aires-real-estate-1.csv.
frame1 = wrangle("./data/buenos-aires-real-estate-1.csv")
print(frame1.info())
frame1.head()

In [ ]:
# Task 2.2.2: Add to the wrangle function below so that, in the DataFrame it returns, the "lat-lon" column is replaced by separate "lat" and "lon" columns. 
# Don't forget to also drop the "lat-lon" column. 
# Be sure to rerun all the cells above before you continue.

def wrangle(filepath):
    # Read CSV file
    df = pd.read_csv(filepath)

    # Subset data: Apartments in "Capital Federal", less than 400,000
    mask_ba = df["place_with_parent_names"].str.contains("Capital Federal")
    mask_apt = df["property_type"] == "apartment"
    mask_price = df["price_aprox_usd"] < 400_000
    df = df[mask_ba & mask_apt & mask_price]

    # Subset data: Remove outliers for "surface_covered_in_m2"
    low, high = df["surface_covered_in_m2"].quantile([0.1, 0.9])
    mask_area = df["surface_covered_in_m2"].between(low, high)
    df = df[mask_area]

    df[["lat", "lon"]] = df["lat-lon"].str.split(",", expand=True)
    df = df.drop("lat-lon", axis="columns")
    
    return df

In [ ]:
# Task 2.2.3: Use you revised wrangle function create a DataFrames frame2 from the file data/buenos-aires-real-estate-2.csv.

frame2 = wrangle("./data/buenos-aires-real-estate-2.csv")

In [ ]:
# Task 2.2.4: Use pd.concat to concatenate frame1 and frame2 into a new DataFrame df. 
# Make sure you set the ignore_index argument to True.
df = pd.concat([frame1,frame2])
print(df.info())
df.head()

In [3]:
# Task 2.2.5: Complete the code below to create a Mapbox scatter plot that shows the location of the apartments in df. 

fig = px.scatter_mapbox(
    df,  # Our DataFrame
    lat="lat",
    lon="lon",
    width=600,  # Width of map
    height=600,  # Height of map
    #color="",
    hover_data=["price_aprox_usd"],  # Display price when hovering mouse over house
)

fig.update_layout(mapbox_style="open-street-map")

fig.show()

NameError: name 'df' is not defined